In [1]:
"""Boundary information for Jefferson County, KY, USA"""

LONGITUDES = LONG_MIN, LONG_MAX = -85.94712712079293, -85.40492183942492
LATITUDES = LAT_MIN, LAT_MAX = 37.99712528351634, 38.38023822809115

DELTA_LONG = LONG_MAX - LONG_MIN  # == 0.5422052813680125
DELTA_LAT = LAT_MAX - LAT_MIN  # == 0.3831129445748118

SOURCE = "../../data/raw/Louisville_Metro_KY_County_Boundaries.geojson"

In [2]:
import json
import os

assert os.path.exists(SOURCE)

The first few lines of the SOURCE file to show the json structure. 

```json
{"type": "FeatureCollection",
"name": "Louisville_Metro_KY_County_Boundaries",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "OBJECTID": 1, "CNTY_NAME": "TRIMBLE", "FIPS": "21223", "STATE_FIPS": "21", "CNTY_FIPS": "223", "SHAPEAREA": 4349563592.2746, "SHAPELEN": 294807.53917238303 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -85.330957285908198, 38.729777516517807 ], [ -85.333144049715258, 38.73595715131929 ], [ -85.333786967475902, 38.735797268561612 ], [ -85.334517444333343, 38.735706030319307 ], [ -85.335569338778541, 38.```

In [3]:
with open(SOURCE, 'r') as file:
    data = json.load(file)

len(data['features'])
# 12. Each feature is a county boundary. Need to find the right one.
counties = data['features']

#counties[11]

Each country feature looks like this.

```json
{'type': 'Feature',
 'properties': {'OBJECTID': 12,
  'CNTY_NAME': 'HARDIN',
  'FIPS': '21093',
  'STATE_FIPS': '21',
  'CNTY_FIPS': '093',
  'SHAPEAREA': 17568447077.2728,
  'SHAPELEN': 770448.545351739},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-85.94710131294255, 38.005040670600735],
    [-85.94712712079293, 38.00507533300107],
    [-85.94842096276824, 38.00478988630253],
    [-85.95015645439582, 38.004582735612516],
    [-85.95154500854562, 38.00455901793901],
```

In [4]:
# The county I am interested in is called Jefferson.


for county in counties:
    if county['properties']['CNTY_NAME'] == 'JEFFERSON':
        JEFFCO = county
        break

coordinates = JEFFCO['geometry']['coordinates'][0]
coordinates

[[-85.57581186859306, 38.33454586847085],
 [-85.57807060364887, 38.335714204871564],
 [-85.5790037713928, 38.336193264774884],
 [-85.58098890773384, 38.33719675866782],
 [-85.58306569784827, 38.33829889828467],
 [-85.58350194850301, 38.3385228219488],
 [-85.58364799522646, 38.33859778747096],
 [-85.58379404218861, 38.33867274937784],
 [-85.5839400895158, 38.338747714533724],
 [-85.58408613708164, 38.33882267607437],
 [-85.58423218501252, 38.33889764086393],
 [-85.58437822888754, 38.33897260552013],
 [-85.5845242773589, 38.339047566511354],
 [-85.58467032619528, 38.339122530751546],
 [-85.58481637527031, 38.33919749137649],
 [-85.58525452443283, 38.33942237901664],
 [-85.5847954957549, 38.33966139548348],
 [-85.58448216641915, 38.33980056848736],
 [-85.5840772599845, 38.339880422522235],
 [-85.58390815313928, 38.339913772368796],
 [-85.58373904607546, 38.33994711853855],
 [-85.58356994327698, 38.33998046784603],
 [-85.5834008359653, 38.340013816958496],
 [-85.58323172849815, 38.34004716

In [5]:

def sort_long_lat(coordinates):
    longitudes, latitudes = set(), set()
    for long, lat in coordinates:
        longitudes.add(long)
        latitudes.add(lat)
    return longitudes, latitudes

longitudes, latitudes = sort_long_lat(coordinates)

blong_max = max(longitudes)
blong_min = min(longitudes)
blat_max = max(latitudes)
blat_min = min(latitudes)

In [6]:
# Get similar data from centerlines to see if any of the roads exit county boundaries. 

CENTERLINES = "../../data/cleaner/centerlines_raw.csv"
assert os.path.exists(CENTERLINES)

import pandas as pd
cl = pd.read_csv(CENTERLINES, low_memory=False)
coords = cl['COORDINATES'].to_list()


In [7]:

#coords[0]
# Each cooordinate is a string.
# Like this: '-85.6809503218278,38.1588670887491 -85.6810007757001,38.158161039538 -85.681234634906,38.1581804844049'

list(map(float, coords[4].split()[0].split(','))) # Works

def get_points():
    for coordinate_string in coords:
        for point in coordinate_string.split():
            yield map(float, point.split(','))

longitudes, latitudes = sort_long_lat(get_points())


clong_max = max(longitudes)
clong_min = min(longitudes)
clat_max = max(latitudes)
clat_min = min(latitudes)

In [8]:
# Some centerlines pass out of the county. 
# I will ignore them for now.

blong_max >= clong_max

False

In [9]:
# These values are all fine.

blong_min <= clong_min
blat_max >= clat_max
blat_min <= clat_min

blong_max

-85.40492183942492

In [10]:
# All tests pass

assert LONG_MAX == blong_max
assert LONG_MIN == blong_min
assert LAT_MAX == blat_max
assert LAT_MIN == blat_min